# CNN on CIFAR10

In [9]:
import torch
import torchvision
import torchsummary

## General Constants

In [22]:
BATCH_SIZE = 128
LEARNING_RATE = 0.001
N_EPOCHS = 30
PATH = '.cifar_model.pth'

## Loading the data

In [4]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform,
)
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform,
)
test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=BATCH_SIZE,
    shuffle=True
)

100.0%


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## The Model

In [15]:
class Model(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(
            in_channels=3,
            out_channels=32,
            kernel_size=5,
            padding='same',
            stride=1,
        )
        self.relu = torch.nn.ReLU()
        self.max_pool = torch.nn.MaxPool2d(2)
        self.flatten = torch.nn.Flatten()
        self.dense = torch.nn.Linear(
            in_features=8192,
            out_features=10
        )
        self.softmax = torch.nn.Softmax()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.softmax(x)
        return x

In [16]:
model = Model()

torchsummary.summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]           2,432
              ReLU-2           [-1, 32, 32, 32]               0
         MaxPool2d-3           [-1, 32, 16, 16]               0
           Flatten-4                 [-1, 8192]               0
            Linear-5                   [-1, 10]          81,930
           Softmax-6                   [-1, 10]               0
Total params: 84,362
Trainable params: 84,362
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.63
Params size (MB): 0.32
Estimated Total Size (MB): 0.96
----------------------------------------------------------------


/tmp/ipykernel_29998/1797031239.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


In [18]:
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [21]:
for epoch in range(N_EPOCHS):
    training_cost = 0.0
    training_correct = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        predictions = model(inputs)
        cost = loss_func(predictions, targets)
        cost.backward()
        optimizer.step()
        training_cost += cost.item()
        training_correct += (torch.argmax(predictions, dim=1) == targets).sum()
    training_cost /= len(train_set)
    training_correct /= len(train_set)
    print(f'epoch: {epoch}, loss: {training_cost}, accuracy: {training_correct}')

/tmp/ipykernel_29998/1797031239.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


epoch: 0, loss: 0.015152500298023223, accuracy: 0.5254200100898743
epoch: 1, loss: 0.014814255061149598, accuracy: 0.570360004901886
epoch: 2, loss: 0.014584864485263825, accuracy: 0.6008800268173218
epoch: 3, loss: 0.014407417750358581, accuracy: 0.6222400069236755
epoch: 4, loss: 0.014242905604839325, accuracy: 0.6442199945449829
epoch: 5, loss: 0.01412603346824646, accuracy: 0.6594799757003784
epoch: 6, loss: 0.01400447177886963, accuracy: 0.6757400035858154
epoch: 7, loss: 0.013902996079921723, accuracy: 0.6888200044631958
epoch: 8, loss: 0.013837937383651733, accuracy: 0.6976600289344788
epoch: 9, loss: 0.01375082513332367, accuracy: 0.7095400094985962
epoch: 10, loss: 0.013692607021331788, accuracy: 0.716920018196106
epoch: 11, loss: 0.013638248076438904, accuracy: 0.7240399718284607
epoch: 12, loss: 0.013603702394962312, accuracy: 0.7283999919891357
epoch: 13, loss: 0.013522961864471435, accuracy: 0.7392399907112122
epoch: 14, loss: 0.013490196342468262, accuracy: 0.743979990482

## Save the trained model

In [23]:
torch.save(model.state_dict(), PATH)

## Load the trained model

In [24]:
model = Model()
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

## Testing the model

In [25]:
correct = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        predictions = model(inputs)
        torch.argmax(predictions)
        correct += (torch.argmax(predictions, dim=1) == targets).sum()

print(f'Accuracy on training set: {correct / len(test_set)}')

/tmp/ipykernel_29998/1797031239.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


Accuracy on training set: 0.657800018787384
